## Load dependencies

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import time
import os
import sys
path = os.path.abspath('../')
print(f"The project folder path is {path}")
sys.path.append(path)
from demand.entities.pointofinterest import PointOfInterestCollection
from demand.handlers.populationdatahandler import PopulationDataHandler
from demand.demand.demand import Demand
from demand.utils import setup_logger, generate_dataset_id, create_bubble_map

## Data config
Edit the below cell to point to the correct data
* `country_code` of the desired data
* pointofinterest is the name of the file containing the pointofinterest , it is located in `/data/country_code/input`

**Example:**
```python 
country_code = 'ESP'
dataset_id = 'ESP-1697915895-xs2u-pointofinterest.csv'
```

In [ ]:
country_code = 'ESP'
poi_dataset_id = 'ESP-1697915895-xs2u-pointofinterest.csv'

In [ ]:
poi_filepath = f'{path}/data/{country_code}/input/{poi_dataset_id}'
log_folder = f'{path}/logs'
population_data_folder = f'{path}/data/population'

## Analysis config

In [ ]:
radii = [1, 2, 3]
radius_for_demand = 1
dataset_year = 2020
one_km_res = True
un_adjusted = True
overlap_allowed = False
mbps_demand_per_user = 5
are_poi_schools = False

## Helpers

In [ ]:
# Set up a logger
logger = setup_logger(f'pcd_{country_code}', log_filepath=f'{log_folder}/{country_code}-{round(time.time())}-pcd.log')

In [ ]:
# Set up the population data handler
population_data_handler = PopulationDataHandler(
    data_dir = population_data_folder,
    country_code = country_code,
    dataset_year = dataset_year,
    one_km_res = one_km_res,
    un_adjusted = un_adjusted,
    logger = logger)

## Load data

In [ ]:
# Load point of interest data as records
entity_records = pd.read_csv(poi_filepath).to_dict('records')

# Load population data into a collection
pointofinterest_collection = PointOfInterestCollection()
pointofinterest_collection.load_from_records(entity_records)

## Initialize `Demand` class

In [ ]:
demand = Demand(
    points_of_interest = pointofinterest_collection,
    population_data_handler = population_data_handler,
    radii = radii,
    logger = logger,
    overlap_allowed = overlap_allowed,
    mbps_demand_per_user = mbps_demand_per_user,
    are_poi_schools = are_poi_schools)

## Perform Analysis

In [ ]:
demand.perform_analysis()

In [ ]:
results = demand.get_results_table()

In [ ]:
results.head()

## Charts

In [ ]:
lat = pointofinterest_collection.get_lat_lon_pairs()[:,0]
lon = pointofinterest_collection.get_lat_lon_pairs()[:,1]
points = gpd.GeoDataFrame(results[['poi_id','number_of_users','total_mbps']].set_index('poi_id'), geometry=gpd.points_from_xy(lon, lat, crs="EPSG:4326"))

In [ ]:
create_bubble_map(points,"total_mbps","blue")

## Save Results

In [ ]:
dataset_id = generate_dataset_id(prefix=country_code)
output_folder = f'{path}/data/{country_code}/output/'
os.makedirs(output_folder, exist_ok=True)
results.to_json(f'{output_folder}/{dataset_id}-demand.json')